## Use Case

This notebook reads downloaded GRIB files and join weather attributes by location to each station.


Main function: 

To use: 


In [13]:
import pandas as pd
import pygrib
import os
import numpy as np
! pip install cfgrib

import cfgrib

! pip install xarray
import xarray as xr

  Obtaining dependency information for xarray from https://files.pythonhosted.org/packages/94/28/3a6365e45721c7c9078968ed94b4a60076bc31d73b8519021a69b4995b63/xarray-2024.9.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.0 MB/s eta 0:00:00:00:010:01


In [3]:
# set path: input, output

input_dir = '../../data/GRIB'

station_file = 'station.csv'


In [17]:
# Read GRIB file
def read_grib_file(file_path):
	grbs = pygrib.open(file_path)
	for grb in grbs:
		print(grb)
	grbs.close()
	
	return grbs

test_input_grib = '../../data/GRIB/2022/era5_land_20220101_0000.grib'
grbs = read_grib_file(test_input_grib)


1:2 metre temperature:K (instant):regular_ll:surface:level 0:fcst time 24 hrs:from 202112310000
2:10 metre U wind component:m s**-1 (instant):regular_ll:surface:level 0:fcst time 24 hrs:from 202112310000
3:10 metre V wind component:m s**-1 (instant):regular_ll:surface:level 0:fcst time 24 hrs:from 202112310000
4:Total precipitation:m (accum):regular_ll:surface:level 0:fcst time 23-24 hrs (accum):from 202112310000


In [22]:

with xr.open_dataset(test_input_grib) as ds:
	df = ds.to_dataframe()

# df.head(30)

# df.to_csv('test.csv')

# in df, delete column: number, time, step, surfcace, valid_time

df = df.drop(columns=['number', 'time', 'step', 'surface', 'valid_time'])

# df.head(30)
date = test_input_grib.split('_')[2]
time = test_input_grib.split('_')[3].split('.')[0]

# add date and time to df

df['date'] = date 
df['time'] = time

df.head(30)



t2m       u10       v10            tp      date  \
latitude longitude                                                           
31.032   72.475     281.226135  0.563780  0.070322  8.521523e-07  20220101   
         72.575     281.187073  0.665342 -0.079092  8.521523e-07  20220101   
         72.675     281.140198  0.788633 -0.137197  8.521523e-07  20220101   
         72.775     280.931213  0.931211 -0.107412  8.521523e-07  20220101   
         72.875     280.493713  1.059385 -0.043203  8.521523e-07  20220101   
         72.975     280.165588  1.130430  0.016611  8.521523e-07  20220101   
         73.075     279.905823  1.202452  0.061289  8.521523e-07  20220101   
         73.175     279.507385  1.328917  0.115488  8.521523e-07  20220101   
         73.275     279.257385  1.424376  0.125742  8.521523e-07  20220101   
         73.375     279.255432  1.446348  0.033457  8.521523e-07  20220101   
         73.475     279.312073  1.433653 -0.105947  8.521523e-07  20220101   
         73.575     279.366760  1.397276 -0.268057  8.521523e-07  20220101   
         73.675     279.282776  1.417051 -0.396231  8.521523e-07  20220101   
         73.775     279.315979  1.387510 -0.487295  8.521523e-07  20220101   
         73.875     279.353088  1.357237 -0.575186  8.521523e-07  20220101   
         73.975     279.567932  1.249327 -0.604239  8.521523e-07  20220101   
         74.075     279.898010  1.085264 -0.589346  8.521523e-07  20220101   
         74.175     280.384338  0.952940 -0.554922  8.521523e-07  20220101   
         74.275     280.474182  0.896544 -0.555410  8.521523e-07  20220101   
         74.375     280.292542  0.845274 -0.590567  8.521523e-07  20220101   
         74.475     280.118713  0.790342 -0.599600  8.521523e-07  20220101   
         74.575     279.993713  0.689512 -0.627920  8.521523e-07  20220101   
         74.675     280.101135  0.527647 -0.691153  8.521523e-07  20220101   
         74.775     280.179260  0.348204 -0.768545  8.521523e-07  20220101   
         74.875     280.134338  0.300840 -0.801260  8.521523e-07  20220101   
         74.975     280.317932  0.298399 -0.780264  8.521523e-07  20220101   
         75.075     280.507385  0.317198 -0.693350  8.521523e-07  20220101   
         75.175     280.612854  0.362852 -0.642324  8.521523e-07  20220101   
         75.275     280.599182  0.512266 -0.647451  8.521523e-07  20220101   
         75.375     280.112854  0.723692 -0.685293  8.521523e-07  20220101   

                    time  
latitude longitude        
31.032   72.475     0000  
         72.575     0000  
         72.675     0000  
         72.775     0000  
         72.875     0000  
         72.975     0000  
         73.075     0000  
         73.175     0000  
         73.275     0000  
         73.375     0000  
         73.475     0000  
         73.575     0000  
         73.675     0000  
         73.775     0000  
         73.875     0000  
         73.975     0000  
         74.075     0000  
         74.175     0000  
         74.275     0000  
         74.375     0000  
         74.475     0000  
         74.575     0000  
         74.675     0000  
         74.775     0000  
         74.875     0000  
         74.975     0000  
         75.075     0000  
         75.175     0000  
         75.275     0000  
         75.375     0000

In [2]:
# Convert GRIB file to dataframe

# grib has 4 channel, for each channel, we need to extract the data and save it to the dataframe




	# grib contains 4 arrarys, export 4 2-d array
	# grbs.select(name='2 metre temperature')[0].values
	# grbs.select(name='2 metre temperature')[0].latlons()
	# grbs.select(name='2 metre temperature')[0].values
	# grbs.select(name='2 metre temperature')[0].values

my_param = '2t','10u','10v','tp'

# select date and hour from file name
my_date = test_input_grib.split('_')[2]
my_time = test_input_grib.split('_')[3].split('.')[0]

print(my_date, my_time)



DATA = np.array(grbs.select(marsParam=my_param,dataDate=my_date,dataTime=my_time))

    # DATA will contain 3 arrays
    # DATA[0] for 2 metre temperature
    # DATA[1] for 10 metre U wind component
    # DATA[2] for 10 metre V wind component
	# DATA[3] for Total precipitation



	# convert np array to dataframe
df = pd.DataFrame(DATA[0].values)
df.columns = ['2t']
df['10u'] = DATA[1].values
df['10v'] = DATA[2].values
df['tp'] = DATA[3].values





: 

In [25]:
# check how many grids are there in df


# check index

df.index

# count number of unique latitudes
# lat_count = df['latitude'].nunique()
# lon_count = df['longitude'].nunique()

# print('lat_count:',lat_count,'lon_count:', lon_count)

# print('pixel number:',lat_count*lon_count)


MultiIndex([(31.032,            72.475),
            (31.032, 72.57499999999999),
            (31.032, 72.67499999999998),
            (31.032, 72.77499999999998),
            (31.032, 72.87499999999997),
            (31.032, 72.97499999999997),
            (31.032, 73.07499999999996),
            (31.032, 73.17499999999995),
            (31.032, 73.27499999999995),
            (31.032, 73.37499999999994),
            ...
            (12.431, 93.87499999999878),
            (12.431, 93.97499999999877),
            (12.431, 94.07499999999877),
            (12.431, 94.17499999999876),
            (12.431, 94.27499999999876),
            (12.431, 94.37499999999875),
            (12.431, 94.47499999999874),
            (12.431, 94.57499999999874),
            (12.431, 94.67499999999873),
            (12.431,            94.775)],
           names=['latitude', 'longitude'], length=41888)

In [9]:
date = test_input_grib.split('_')[2]
print(date)

time = test_input_grib.split('_')[3].split('.')[0]
print(time)

param = '2t','10u','10v','tp'


20220101
0000


In [44]:
# read station csv

station_file='../../data/STATION.csv'
station_df = pd.read_csv(station_file)

station_df.head(30)

# print lat and lon data type
# print(station_df['Lat'].dtype)


,station_id,Location name,ESMI_ID,From date,To date,District,State,Category,Connection Type,Lat,Lon
0,1,5th phase JP Nagar,NaN,1/30/2017 0:00,6/12/2017 12:59,Bengaluru Urban,Karnataka,State Capital,Domestic,12.901092,77.589150
1,2,80 feet road,NaN,9/6/2015 1:00,10/2/2018 14:59,Dhule,Maharashtra,District Headquarters,Domestic,20.895199,74.775982
2,3,Adarsh Nagar,NaN,3/2/2016 0:00,1/22/2017 20:59,Saharsa,Bihar,District Headquarters,Domestic,25.883507,86.614919
3,4,Adgaon,NaN,5/4/2018 0:00,12/31/2018 23:59,Nashik,Maharashtra,Gram Panchayat,Domestic,20.037184,73.850136
4,5,Agarchitti,NaN,1/1/2017 0:00,6/12/2017 12:59,Chamoli,Uttarakhand,Gram Panchayat,Domestic,30.016116,79.308735
5,6,Agarwal Mohalla Jarwal Kasba,NaN,1/26/2018 0:00,12/31/2018 23:59,Bahraich,Uttar Pradesh,Other Municipal Area,Domestic,27.574566,81.601251
6,7,Agri road Chidambaram,NaN,4/28/2018 0:00,12/31/2018 23:59,Cuddalore,Tamil Nadu,Other Municipal Area,Domestic,11.390684,79.721755
7,8,Aishbagh,NaN,11/3/2017 0:00,10/14/2018 9:59,Lucknow,Uttar Pradesh,State Capital,Domestic,26.841267,80.903863
8,9,Ajekar,NaN,1/9/2015 17:00,5/15/2015 2:59,Udupi,Karnataka,Gram Panchayat,Domestic,13.320509,74.998807
9,10,Akoli Jehangir,NaN,10/31/2014 23:00,3/8/2015 0:59,Akola,Maharashtra,Gram Panchayat,Domestic,21.154837,77.118466


In [51]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree

# Assuming df (grib data) has lat and lon as index
# And the columns are ['t2m', 'wind_u', 'wind_v']
# Station_df has ['station_lat', 'station_lon']

# Step 1: Extract the grid coordinates from the grib data
grid_points = np.array(list(zip(df.index.get_level_values('latitude'), df.index.get_level_values('longitude'))))

# Step 2: Build the KDTree
tree = KDTree(grid_points)

# Step 3: For each station coordinate, find the nearest grid point
station_coords = station_df[['Lat', 'Lon']].to_numpy()

# Query the nearest point in the KDTree
_, nearest_grid_idx = tree.query(station_coords)

# Step 4: Use the index to get the corresponding grid data
nearest_grid_points = grid_points[nearest_grid_idx]
print(nearest_grid_points)


KeyError: 'Level latitude not found'

In [49]:
df.index = pd.MultiIndex.from_tuples(
    [(round(lat, 3), round(lon, 3)) for lat, lon in df.index], 
)

# in df, round to latitude and longitute to 3


<bound method NDFrame.head of                       t2m       u10       v10            tp      date  time
31.032 72.475  281.226135  0.563780  0.070322  8.521523e-07  20220101  0000
       72.575  281.187073  0.665342 -0.079092  8.521523e-07  20220101  0000
       72.675  281.140198  0.788633 -0.137197  8.521523e-07  20220101  0000
       72.775  280.931213  0.931211 -0.107412  8.521523e-07  20220101  0000
       72.875  280.493713  1.059385 -0.043203  8.521523e-07  20220101  0000
...                   ...       ...       ...           ...       ...   ...
12.431 94.375         NaN       NaN       NaN           NaN  20220101  0000
       94.475         NaN       NaN       NaN           NaN  20220101  0000
       94.575         NaN       NaN       NaN           NaN  20220101  0000
       94.675         NaN       NaN       NaN           NaN  20220101  0000
       94.775         NaN       NaN       NaN           NaN  20220101  0000

[41888 rows x 6 columns]>

In [50]:

nearest_grid_values = df.loc[nearest_grid_points[:, 0], nearest_grid_points[:, 1]].reset_index()


# Step 5: Combine the nearest grid data with station data
# station_df = pd.concat([station_df.reset_index(drop=True), nearest_grid_values[['t2m', 'wind_u', 'wind_v']]], axis=1)
# preserve all df columns in nearest_grid_values
# The station_df now contains the nearest grid point attributes (t2m, wind_u, wind_v) for each station.


KeyError: "None of [Index([ 77.5749999999997, 74.77499999999986, 86.57499999999919,\n       73.87499999999991, 79.27499999999961, 81.57499999999948,\n       79.67499999999959, 80.87499999999952, 74.97499999999985,\n       77.07499999999973,\n       ...\n       84.27499999999932, 84.97499999999928, 87.37499999999915,\n       84.27499999999932, 85.27499999999927, 85.37499999999926,\n       85.57499999999925, 90.57499999999897,  91.6749999999989,\n       78.97499999999962],\n      dtype='float64', length=538)] are in the [columns]"